This is an [jupyter](http://jupyter.org) notebook.
Lectures about Python, useful both for beginners and experts, can be found at http://scipy-lectures.github.io.

Open the notebook by (1) copying this file into a directory, (2) in that directory typing 
jupyter-notebook
and (3) selecting the notebook.

# <font color= 'blue'>Blind Source Separation</font>
## Independent Component Analysis

***
A notebook by ***Shashwat Shukla*** and ***Dhruv Ilesh Shah***
***

In this tutorial we will learn how to solve the Cocktail Party Problem using Independent Component Analysis(ICA).
We will first take a look at Principle Component Analysis(PCA). The limitations of PCA will naturally lead to an understanding of ICA does.

# Overview
## The Cocktail Party Problem(CPP)

So what is the Cocktail Party Problem? 
Imagine you are at a party where a lot of different conversations are happening in different parts of the room. As a listener in  the room, you are receiving sound from all of these conversations at the same time. And yet, as humans, we possess the ability to identify different threads of conversation and to focus on any conversation of our choice. How do we do that? And how can we program a computer to do that?
So this is essentially the Cocktail Party Problem: Given **m** sources(conversations at the party for example), and some number of sound receivers, separate out the different signals. (We will talk about how many receivers we need later.)

We need to make some mathematical assumptions and also phrase the problem more formally.

## The data

So first of all, our signals here are the sounds coming from different sources. 
At every (uniformly spaced) discrete interval of time we record **m** samples, one at each of our **m** microphones.

Note the implicit assumptions that we have made here:

1) There are as many microphoneses as there are independent conversations(sources) going on in the room. This assumption allows us to come up with a method to retrieve all the m independent signals. We can say that our system is **critically determined**(and is not under- or over- determined). Henceforth, we shall only consider this case in the tutorial.

2) Each microphone records a reasonably distinct combination of the independent signals. This simply amounts to not keeping two microphones too close to each other. Due to practical computational limits (see floating point math), it is always best to have easily distinguishable recordings. 

How are we recording this data? We simply record the amplitude of the sound at each instant. Recording the pressure amplitude is a convenient thing to do(and is what a microphone does. A transducer then converts the pressure amplitude to a voltage).
Note that we are recording the signals at discrete intervals of time (at a rate assumed to be greater than the Shannon rate)and will be working only in the time domain with these discrete signals.

One very important thing: We assume that the sound that any receiver records is a **linear combination** of sounds from the different sources. This is a reasonable assumption to make as pressure adds linearly. Each receiver will receive a different linear combination: If the first receiver is closer to a particular speaker than the second receiver, then the linear weight of this speaker will be proportionately higher for the first receiver.

![Cocktail Party Problem](Notebook/cocktail_1.png)


We further assume that each source is **statistically independent** with respect to all the other sources. We will look at a mathematical interpretation of statistical independence of two signals later. Within the context of the Cocktail Party parable an intuitive understanding of this assumption follows naturally, as the conversations happening in different parts of the room are independent of each other. Hence, knowing the signal at a particular instant from one source does not allow us to predict the value of the signal from any other source at that instant. They are independent variables.

This is the key assumption in Blind Source separation that allows to solve the problem.

We are also making one vital assumption about the sources of the signals: that they are non-Gaussian. We will look at what that means and why it matters in the next section. 

## The math

We will index our microphones from **1** to **m**. 

The signal received by the microphone labelled **i** over the entire time of recording will be denoted by $x_{i}$. A particular sample of this recorded signal, recorded at the time index **j** will then be denoted by $x_{i}^{j}$. 

Hence, if the samples of the signals recorded over time be **N**, then $x_{i}$ can be seen to be a row vector in **N**-dimensional space. It's jth element is given by $x_{i}^{j}$.

We had said that we have **m** microphones. Hence **i** in the above description ranges from **1** to **m**.

If we stack up these row vectors, we will get an **m x N** matrix whose ith row corresponds to the samples recorded by a particular microphone. A 'vertical slice' of this matrix, i.e a column corresponds to all the samples recorded at a given instant of time, indexed by the indices of the corresponding microphone.

Let us call this data matrix **x**.

To reiterate, $x_{i}^{j}$ corresponds to the sound sample recorded by the **i**th mike at the time (indexed by) **j**. 

Let us now similarly define matrices corresponding to the sources that we wish to finally recover.
The indices for the independent sources also go from **1** to **m**.

Let $s_{i}$ denote the signal generated by the **i**th independent source that we wish to recover (the **i**th conversation in the room). It is defined as a row vector.

$s_{i}^{j}$ is then the **j**th time sample of this signal. 

Again, we vertically stack up these row vectors to get a **m x N** matrix denoted by **s**.

Now that we have defined our data and the signals that we wish to retrieve, we will describe the (assumed) relationship between the two. Note that we had assumed that the independent sources add **linearly** to give the recorded signals. 

This means that each $x_{i}$ is some linear combination of the vectors $s_{1}$ through $s_{m}$.

Putting it all together, we conclude that $x = As$ ; for some **m x m** matrix **A**, called the mixing matrix.

Our objective is to then find an "un-mixing" matrix **W** that satisfies $s = Wx$.

If we know this **W**, as we already have **x**, we can calculate **s** by a direct multiplication. 

![The Problem](Notebook/cocktail_2.jpg)


## Outline of solution

Our objective is to find the matrix **W**. As we have assumed that the number of microphones is equal to the number of independent conversations, it turns out that the matrix **A** is invertible and hence **W** is just the inverse of **A**. \

Hence, it suffices to find **A**. We will employ [Singular Value Decomposition(SVD)](https://www.wikiwand.com/en/Singular_value_decomposition) on the matrix **A**.

Hence, $A = UDV$ for orthogonal matrices **U**, **V** and diagonal matrix **D**. 

We will then determine each of **U**, **D**, **V** by considering the covariance matrix of **x** and exploiting the independence of the source signals.

The details follow.
***

## <font c>Covariance

An important term in the concept of statistics is *covariance*, which is a measure of how much two random variables change _together_. Covariance provides a measure of the strength of the correlation between two or more sets of random variates. The covariance for two random variates X and Y, each with sample size N, is defined by the expectation value:
$$ cov (X,Y) = \langle(X-\mu_x) (Y-\mu_y)\rangle \\
=\langle X \rangle \langle Y \rangle - \mu_x \mu_y$$
where $\mu_x = \langle X \rangle$ and $\mu_y = \langle Y \rangle$ are the respective means.  
For uncorrelated variates, $ \langle X Y \rangle = \langle X \rangle \langle Y \rangle $ and hence,
$$ cov(X,Y) =  \langle X Y \rangle - \mu_x \mu_y = \langle X \rangle \langle Y \rangle - \mu_x \mu_y = 0 $$
However, if the variables are correlated in some way, then their covariance will be nonzero. In fact, if $cov(X,Y)>0$, then Y tends to increase as X increases, and if $cov(X,Y)<0$, then  Y tends to decrease as X increases. Note that while statistically independent variables are always uncorrelated, the converse is not necessarily true.

As you can see, covariance can be a good metric to analyse the dependence of two datasets. To read more about covariance, follow [this link.](http://mathworld.wolfram.com/Covariance.html)

As a special case, substituting $ X = Y $  gives $$cov(X,X) = \langle X^2 \rangle - \langle X \rangle ^2 \\ =\sigma^2_X$$
where $\sigma_X$ denotes the _standard deviation_. Thus, $cov(X,Y)$ reduces to the statistical variance for this case.


Given a dataset vector $X$ _(nx1 vector)_, the covariance of $X$ is given by $$ C_x= (X-\mu_x)(X-\mu_x)^T $$
This matrix $C_x$ has very interesting properties, which we will be exploiting in the upcoming sections. This [document](http://www.robots.ox.ac.uk/~davidc/pubs/tt2015_dac1.pdf) would be an interesting read.
***

Now that we have the necessary tools required, let us dive into some algorithms that involve manipulating the information matrix and separating them into components. The first such algorithm is the PCA.
## Principle Component Analysis

_Principal Component Analysis (PCA)_ is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components. The number of principal components is less than or equal to the number of original variables. This transformation is defined in such a way that the first principal component has the **largest possible variance** (that is, accounts for as much of the variability in the data as possible), and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal to the preceding components.

What does this mean? Let's visualise the problem at hand.  
Let us say you are given a dataset of point in the two-dimensional plane like the plot given below. ![](https://cmp.felk.cvut.cz/cmp/software/stprtool/examples/pca_example.gif)
The given dataset is two-dimensional and has a set of points arranged in a roughly elliptical form. What the PCA algorithm does is that it identifies the first principle component of the data, that is, the direction along which the _variance_ of the data is maximum. This is the direction indicated by the red line. The second principle component is the one orthogonal to this, along the approximate minor axis of the dataset. Another such example of PCA on a 2D dataset is shown below. ![](http://www.visiondummy.com/wp-content/uploads/2014/05/correlated_2d.png)
Here, the green line shows the vector corresponding to the first principle component, and pink shows the second component. To visualise how PCA can be applied on higher dimensional datasets, check out [this page](http://setosa.io/ev/principal-component-analysis/).

The PCA has various applications, some of which have been listed below.
* To identify the most important feature(s) of a multivariate function.
* Reduce dimensionality of data for saving memory, preserving maximum information.
* Speed up processes (regression, for example) by ignoring the features with low variance.
* As a pre-processing technique for further statistical methods, increasing efficiency.

Before we go ahead with understanding PCA, it is important to consider the following ** assumptions and limitation ** of PCA:
1. PCA assumes the dataset to be a linear combination of the variables.
2. There is no guarantee that the directions of maximum variance will contain good features for discrimination.
3. PCA assumes that components with larger variance correspond to interesting dynamics and lower ones correspond to noise.
4. Output vectors of PCA are orthogonal, which means that the principal components are _orthogonal_ to each other.
5. PCA requires the data to be _mean normalized_ and _univariate_, as it is highly susceptible to unscaled variables.
6. Since it assumes data to be uncorrelated and accurate, it is _vulnerable to outliers_ and hence may produce incorrect results.

Here, we see that PCA breaks down the dataset into uncorrelated (hence, orthogonal) components. 
 ### <u>The Algorithm </u>

Principle component analysis relies on the property that the eigenvectors of the covariance matrix for a dataset $X$ represent a new set of orthogonal components that are the principle components of the dataset. Mathematically, given the covariance matrix $C_x$, the matrix $U = eig(C_x)$ which returns the eigenvectors is the desired matrix.  
Another method to do this is *singular value decomposition* of the covariance matrix. In linear algebra, the singular value decomposition (SVD) is a factorization of a real or complex matrix. It is the generalization of the eigendecomposition of a positive semidefinite normal matrix (for example, a symmetric matrix with positive eigenvalues) to any $m \times n$ matrix via an extension of polar decomposition. The theory behind SVD is exhaustive and beyond the scope of this tutorial; you can find some interesting stuff [here.](http://web.mit.edu/be.400/www/SVD/Singular_Value_Decomposition.htm) For our use, SVD is called in a programming language like Matlab or Python as $$ [U, S, V] = svd(C_x) $$
Such that $C_x$ satisfies $ C_x = USV^T $. Note that $U$ here refers to the same matrix of orthogonal eigenvectors.

Thus, $U$ here is an $n\times n$ matrix with its column vectors as the eigenvectors of $C_x$.
* If we aim to perform data compression, define a matrix $U_{reduced} = U[:,1:k]$ which stores the first $k$ principle components. Thus, the required dataset with reduced dimensionality $k$ can be given by $X_{new} = U_{reduced}^TX$.
* If we do not want to compress data but simply express it in terms of principle/orthogonal components, we have $X_{new} = U^TX$